In [49]:
import os
import tensorflow as tf
from PIL import Image
import PIL
import numpy as np

In [50]:
folder_path = os.getcwd()
ws_path = os.path.dirname(folder_path)
ws_path

'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification'

In [51]:
list_dir = os.listdir(ws_path)
frame_path = ws_path + '/rawFrame'
prepro_data_path = ws_path + '/preproDataset'
nor_fr_path = frame_path + '/normal'
vsd_fr_path = frame_path + '/vsd'
nor_prepro_path = prepro_data_path + '/normal'
vsd_prepro_path = prepro_data_path + '/vsd'

In [52]:
# Create output directory
os.makedirs(prepro_data_path, exist_ok=True)
os.makedirs(nor_prepro_path, exist_ok=True)
os.makedirs(vsd_prepro_path, exist_ok=True)

In [53]:
nor_img = os.listdir(nor_fr_path)
nor_fr_list_path = list()

for img in nor_img:
    path = nor_fr_path + f'/{img}'
    nor_fr_list_path.append(path)

# nor_fr_list_path

In [54]:
vsd_img = os.listdir(vsd_fr_path)
vsd_fr_list_path = list()

for img in vsd_img:
    path = vsd_fr_path + f'/{img}'
    vsd_fr_list_path.append(path)

# vsd_fr_list_path

In [55]:
import pandas as pd


df = pd.DataFrame({'path': nor_fr_list_path, 'label': 0})
df = pd.concat([df, pd.DataFrame({'path': vsd_fr_list_path, 'label': 1})])
X = df[['path']]
y = df['label']

In [56]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
x_res, y_res = rus.fit_resample(X, y)
y_res.value_counts()

label
0    337
1    337
Name: count, dtype: int64

In [57]:
df_res = x_res.copy()
df_res['label'] = y_res
normal_paths = df_res[df_res['label'] == 0]['path'].to_list()
vsd_paths = df_res[df_res['label'] == 1]['path'].to_list()

In [58]:
def preprocess(input_path, output_path, img_size=(224, 224)):
    for path in input_path:
        img = tf.keras.preprocessing.image.load_img(path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)

        img_array = img_array / 255.0
        img_uint8 = tf.image.convert_image_dtype(img_array, dtype=tf.uint8)

        filename = os.path.basename(path)
        filename = os.path.splitext(filename)[0] + '.png'

        output_file = os.path.join(output_path, filename)

        tf.keras.utils.save_img(output_file, img_uint8)


In [59]:
preprocess(normal_paths, nor_prepro_path)
preprocess(vsd_paths, vsd_prepro_path)